# Run a Periodogram Across Full ZTF Sources

This notebook is an adaptation of the Nested Dask [tutorial for loading HiPSCat data](https://nested-dask.readthedocs.io/en/latest/tutorials/work_with_lsdb.html).

There we loaded a small subset of ZTF DR 14, and here we load the full ZTF DR 14 and run a `light-curve` package periodogram across all of ZTF on PSC Bridges2.


Performance spreadsheet
https://docs.google.com/spreadsheets/d/19-GexwAu1TBunGKkCNMU7c3uwhbcLwZXteLLvT3Q48w/edit?usp=sharing

## Install dependencies for the notebook

The notebook requires `nested-dask` and few other packages to be installed.
- `lsdb` to load and join "object" (pointing) and "source" (detection) ZTF catalogs
- `aiohttp` is `lsdb`'s optional dependency to download the data via web
- `light-curve` to extract features from light curves
- `matplotlib` to plot the results

In [1]:
# Uncomment the following line to install nested-dask
# %pip install nested-dask

# Comment the following line to skip dependencies installation
%pip install --quiet tqdm aiohttp light-curve matplotlib dask-jobqueue

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip uninstall -y --quiet lsdb hipscat
%pip install --quiet git+https://github.com/astronomy-commons/lsdb.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from importlib.metadata import version
from pathlib import Path

import dask.array
import dask.distributed
import dask_jobqueue
import light_curve as licu
import matplotlib.pyplot as plt
import nested_pandas as npd
import numpy as np
import pandas as pd
from lsdb import read_hipscat
from matplotlib.colors import LogNorm
from nested_dask import NestedFrame

In [4]:
print(f"{version('lsdb') = }")
print(f"{version('nested-dask') = }")
print(f"{version('dask') = }")

version('lsdb') = '0.3.1.dev1+ge3600fd'
version('nested-dask') = '0.2.0'
version('dask') = '2024.8.2'


Some additional setup for using Dask on PSC Bridges2:

In [5]:
from tqdm import tqdm
import dask
dask.config.set({
    "temporary-directory": Path(os.environ["LOCAL"]) / "dask",
    'distributed.comm.timeouts.connect': '600s',
    'distributed.comm.timeouts.tcp': '600s',
})

#from dask.diagnostics import ProgressBar
#ProgressBar().register()
# Unclear how we want shuffle compression configured
#dask.config.set({"dataframe.shuffle-compression": 'Snappy'})

## Load ZTF DR14
Here we load a special version of ZTF DR14, where all light-curves are matched to PS1 catalog.
We load the data from HTTPS as two LSDB catalogs: objects (metadata catalog) and source (light curve catalog).

## Setup cone search
Here we use a cone search for the LSDB load to keep our data small

In [6]:
from lsdb.core.search import ConeSearch
# search_area = ConeSearch(ra=254, dec=35, radius_arcsec=32.0 * 3600)
search_area = None

In [7]:
#half_degree_catalogs_dir = "https://epyc.astro.washington.edu/~lincc-frameworks/half_degree_surveys/ztf/"
# catalogs_dir = "https://epyc.astro.washington.edu/~lincc-frameworks/hipscat_surveys"
# catalogs_dir = "/data3/epyc/data3/hipscat/catalogs/ztf_axs"
catalogs_dir = "/ocean/projects/phy210048p/shared/hipscat/catalogs/ztf_axs"


lsdb_object = read_hipscat(
    f"{catalogs_dir}/ztf_dr14",
    columns=["ra", "dec", "ps1_objid"],
    search_filter=search_area,
)
lsdb_source = read_hipscat(
    f"{catalogs_dir}/ztf_zource",
    columns=["mjd", "ra", "dec", "mag", "magerr", "band", "ps1_objid", "catflags"],
    search_filter=search_area,
)
lc_columns = ["mjd", "mag", "magerr", "band", "catflags"]

/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/hipscat/io/file_io/file_io.py:320: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(
/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/hipscat/io/file_io/file_io.py:320: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(


In [8]:
lsdb_source

,mjd,ra,dec,mag,magerr,band,ps1_objid,catflags
npartitions=41679,,,,,,,,
0,double[pyarrow],double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],string[pyarrow],int64[pyarrow],int16[pyarrow]
4503599627370496,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
13833932155375321088,...,...,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...,...,...


We need to merge these two catalogs to get the light curve data.
It is done with LSDB's `.join_nested()` method which would give us a new catalog with a nested frame of ZTF sources. For this tutorial we'll just use the underlying nested dataframe for the rest of the analysis rather than the LSDB catalog directly.

In [9]:
# Nesting Sources into Object
nested_ddf = lsdb_object.join_nested(lsdb_source, left_on="ps1_objid", right_on="ps1_objid", nested_column_name="lc")

# TODO remove once have added LSDB wrappers for nested_dask (reduce, dropna, etc)
nested_ddf = nested_ddf._ddf

/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/lsdb/dask/join_catalog_data.py:334: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


## Convert LSDB joined catalog to `nested_dask.NestedFrame`

First, we plan the computation to convert the joined Dask DataFrame to a NestedFrame.

Now we filter our dataframe by the `catflags` column (0 flags correspond to the perfect observational conditions) and the `band` column to be equal to `r`.
After filtering the detections, we are going to count the number of detections per object and keep only those objects with more than 10 detections.

In [10]:
r_band = nested_ddf.query("lc.catflags == 0 and lc.band == 'r'")
nobs = r_band.reduce(np.size, "lc.mjd", meta={0: int}).rename(columns={0: "nobs"})
r_band = r_band[nobs["nobs"] > 10]
r_band

,ra,dec,ps1_objid,lc
npartitions=41679,,,,
0,double[pyarrow],double[pyarrow],int64[pyarrow],"nested<mjd: [double], ra: [double], dec: [double], mag: [float], magerr: [float], band: [string], catflags: [int16]>"
4503599627370496,...,...,...,...
...,...,...,...,...
13833932155375321088,...,...,...,...
18446744073709551615,...,...,...,...


Later we are going to extract features, so we need to prepare light-curve data to be in the same float format.

### Extract features from ZTF light curves

Now we are going to extract some features:
- Top periodogram peak
- Mean magnitude
- Von Neumann's eta statistics
- Excess variance statistics
- Number of observations

We are going to use [`light-curve`](https://github.com/light-curve/light-curve-python) package for this purposes

In [11]:
%%time

extractor = licu.Extractor(
    licu.Periodogram(
        peaks=1,
        max_freq_factor=1.0, # Currently 1.0 for fast runs, will raise for more interesting graphs later
        fast=True,
    ),  # Would give two features: peak period and signa-to-noise ratio of the peak
)


# light-curve requires all arrays to be the same dtype.
# It also requires the time array to be ordered and to have no duplicates.
def extract_features(mjd, mag, **kwargs):
    # We offset date, so we still would have <1 second precision
    t = np.asarray(mjd - 60000, dtype=np.float32)
    _, sort_index = np.unique(t, return_index=True)
    features = extractor(
        t[sort_index],
        mag[sort_index],
        **kwargs,
    )
    # Return the features as a dictionary
    return dict(zip(extractor.names, features))


features = r_band.reduce(
    extract_features,
    "lc.mjd",
    "lc.mag",
    meta={name: np.float32 for name in extractor.names},
)

CPU times: user 3.44 ms, sys: 0 ns, total: 3.44 ms
Wall time: 3.46 ms


Before we are going next and actually run the computation, let's create a Dask client which would allow us to run the computation in parallel.

Now we can collect some statistics and plot it. 

In [12]:
class Job(dask_jobqueue.slurm.SLURMJob):
    # Rewrite the default, which is a property equal to cores/processes
    worker_process_threads = 3
    
    
class Cluster(dask_jobqueue.SLURMCluster):
    job_cls = Job

    
gb_per_process = 32
jobs = 60
processes = 1  # Single dask worker per slurm job
gb_per_core = 2  # PSC "regular memory" nodes provide fixed 2GB / core
cluster = Cluster(
    # Number of Dask workers per node
    processes=processes,
    # Regular memory node type on PSC bridges2
    queue="RM-shared",
    # dask_jobqueue requires cores and memory to be specified
    # We set them to match RM specs
    cores=gb_per_process // gb_per_core,
    memory=f"{gb_per_process * processes}GB",
    walltime="6:00:00",
)

# processes = 8
# jobs = 3
# cluster = SLURMCluster(
#     # Number of Dask workers per node
#     processes=processes,
#     # Regular memory node type on PSC bridges2
#     queue="RM",
#     # dask_jobqueue requires cores and memory to be specified
#     # We set them to match RM specs
#     cores=128,
#     memory=f"256GB",
#     walltime="1:00:00",
# )

# Run multiple jobs
cluster.scale(jobs=jobs)
# Allow to run more jobs
# cluster.adapt(maximum_jobs=100)
client = dask.distributed.Client(cluster)
# Wait all workers to start
cluster.wait_for_workers(jobs * processes)



# client = dask.distributed.Client(n_workers=8, threads_per_worker=8)



client

<Client: 'tcp://10.8.9.187:38669' processes=47 threads=141, memory=1.37 TiB>

2024-09-05 16:25:00,058 - tornado.application - ERROR - Uncaught exception GET /status/ws (10.8.11.31)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='10.8.11.31')
Traceback (most recent call last):
  File "/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/tornado/websocket.py", line 938, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/tornado/web.py", line 3301, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token 

In [13]:
%%time

# Run the computation
mean_period = features['period_0'].mean().compute()
mean_period

/ocean/projects/phy210048p/malanche/lsdb-tests/cenv/lib/python3.11/site-packages/distributed/client.py:3358: UserWarning: Sending large graph of size 40.23 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 31min 15s, sys: 1min 29s, total: 32min 44s
Wall time: 1h 31min 19s


313.525146328033

In [14]:
client.close()

In [15]:
cluster.close()